<a href="https://colab.research.google.com/github/Chovy-Culver/weatherwise-Chufengsheng-Zheng-doe/blob/main/WeatherWise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
# Create a config.py file with the API key
with open("config.py", "w") as f:
    f.write('API_KEY = "1da7dea923d784a5dd83bac94cde8d65"')


In [61]:
from config import API_KEY

print("Your API Key is:", API_KEY)

Your API Key is: 1da7dea923d784a5dd83bac94cde8d65


In [62]:
%%writefile weather_data/fetch.py
"""
fetch.py - Module for retrieving weather data from OpenWeatherMap API.
"""

import requests
from config import API_KEY

BASE_URL = "https://api.openweathermap.org/data/2.5/"

def fetch_current_weather(city):
    """
    Fetch current weather data for a given city.
    """
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def fetch_forecast_weather(city):
    """
    Fetch 5-day forecast data (3-hour intervals) for a given city.
    """
    url = f"{BASE_URL}forecast?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None


Overwriting weather_data/fetch.py


In [63]:
!mkdir -p weather_data
!touch weather_data/__init__.py


In [64]:
%%writefile weather_data/visualise.py
"""
visualise.py - Module for visualising weather forecast data.
"""

import matplotlib.pyplot as plt
from datetime import datetime

def plot_temperature_trend(forecast_data, city):
    """
    Plot temperature trend from forecast data.
    """
    times = []
    temps = []

    for entry in forecast_data['list']:
        dt = datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S")
        temp = entry['main']['temp']
        times.append(dt)
        temps.append(temp)

    plt.figure(figsize=(10, 5))
    plt.plot(times, temps, marker='o')
    plt.title(f"Temperature Trend in {city}")
    plt.xlabel("Date & Time")
    plt.ylabel("Temperature (°C)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.grid(True)
    plt.show()

def plot_precipitation_probability(forecast_data, city):
    """
    Plot precipitation probability from forecast data (if available).
    """
    times = []
    pop = []

    for entry in forecast_data['list']:
        dt = datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S")
        prob = entry.get('pop', 0) * 100  # pop is probability of precipitation
        times.append(dt)
        pop.append(prob)

    plt.figure(figsize=(10, 5))
    plt.bar(times, pop)
    plt.title(f"Precipitation Probability in {city}")
    plt.xlabel("Date & Time")
    plt.ylabel("Probability (%)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.grid(True)
    plt.show()

def plot_wind_speed(forecast_data, city):
    """
    Plot wind speed trend from forecast data.
    """
    times = []
    speeds = []

    for entry in forecast_data['list']:
        dt = datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S")
        speed = entry['wind']['speed']
        times.append(dt)
        speeds.append(speed)

    plt.figure(figsize=(10, 5))
    plt.plot(times, speeds, marker='s', linestyle='--')
    plt.title(f"Wind Speed Forecast in {city}")
    plt.xlabel("Date & Time")
    plt.ylabel("Wind Speed (m/s)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.grid(True)
    plt.show()


Overwriting weather_data/visualise.py


In [65]:
from weather_data.visualise import (
    plot_temperature_trend,
    plot_precipitation_probability,
    plot_wind_speed
)


In [66]:
pip install requests matplotlib

In [67]:
"""
test_fetch.py - Simple test script for fetch_current_weather and fetch_forecast_weather.
"""

from weather_data.fetch import fetch_current_weather, fetch_forecast_weather

def test_fetch_weather():
    city = "Perth"

    print("🔍 Testing current weather data...")
    current = fetch_current_weather(city)
    if current:
        print("✅ Current weather fetched successfully!")
        print(f"City: {current['name']}")
        print(f"Condition: {current['weather'][0]['description']}")
        print(f"Temperature: {current['main']['temp']}°C")
    else:
        print("❌ Failed to fetch current weather.")

    print("\n🔍 Testing forecast weather data...")
    forecast = fetch_forecast_weather(city)
    if forecast:
        print("✅ Forecast data fetched successfully!")
        print(f"Total data points: {len(forecast['list'])}")
        print(f"First timestamp: {forecast['list'][0]['dt_txt']}")
        print(f"Predicted temperature: {forecast['list'][0]['main']['temp']}°C")
    else:
        print("❌ Failed to fetch forecast weather.")

if __name__ == "__main__":
    test_fetch_weather()


🔍 Testing current weather data...
✅ Current weather fetched successfully!
City: Perth
Condition: few clouds
Temperature: 18.54°C

🔍 Testing forecast weather data...
✅ Forecast data fetched successfully!
Total data points: 40
First timestamp: 2025-05-19 03:00:00
Predicted temperature: 18.54°C


In [2]:
"""
parser.py - Enhanced parser with fuzzy matching for weather intent detection.
"""

from difflib import get_close_matches

INTENT_KEYWORDS = {
    "temperature": ["cold", "hot", "temperature", "jacket", "warm", "cool"],
    "rain": ["rain", "umbrella", "wet", "drizzle"],
    "wind": ["wind", "windy", "breeze", "gust"]
}

# Flatten all keywords to map them to intents
KEYWORD_TO_INTENT = {
    keyword: intent
    for intent, keywords in INTENT_KEYWORDS.items()
    for keyword in keywords
}

def infer_weather_intent(user_input):
    """
    Infer weather intent from input using exact and fuzzy keyword matching.
    Returns one of: "temperature", "rain", "wind", or None.
    """
    user_input = user_input.lower().split()

    for word in user_input:
        # Exact match
        if word in KEYWORD_TO_INTENT:
            return KEYWORD_TO_INTENT[word]
        # Fuzzy match (e.g., 'temprature' -> 'temperature')
        match = get_close_matches(word, KEYWORD_TO_INTENT.keys(), n=1, cutoff=0.8)
        if match:
            return KEYWORD_TO_INTENT[match[0]]

    return None


In [5]:
"""
responder.py - Modular response generator with intent-function mapping.
"""

def handle_temperature(weather_data):
    temp = weather_data['main']['temp']
    desc = weather_data['weather'][0]['description']

    if temp < 10:
        return f"It's quite cold with {temp}°C and {desc}. You might need a coat."
    elif temp < 20:
        return f"The temperature is {temp}°C with {desc}. Maybe bring a light jacket."
    else:
        return f"It's warm today at {temp}°C with {desc}."

def handle_rain(weather_data):
    desc = weather_data['weather'][0]['description'].lower()
    if "rain" in desc or "drizzle" in desc:
        return f"Yes, it looks like rain today: {desc}. Don't forget your umbrella!"
    else:
        return f"No rain expected today. Conditions are {desc}."

def handle_wind(weather_data):
    speed = weather_data['wind']['speed']
    if speed > 10:
        return f"It’s quite windy today with wind speeds around {speed} m/s."
    else:
        return f"Wind is mild today at around {speed} m/s."

# Intent to handler function mapping
INTENT_HANDLERS = {
    "temperature": handle_temperature,
    "rain": handle_rain,
    "wind": handle_wind
}

def generate_response(intent, weather_data):
    """
    Generate a natural language weather response based on user intent.
    """
    if not weather_data:
        return "Sorry, I couldn't get the weather data right now."

    handler = INTENT_HANDLERS.get(intent)
    if handler:
        return handler(weather_data)
    else:
        return "I'm not sure what weather info you need. Please ask about temperature, rain, or wind."
